In [0]:
dbutils.fs.rm("dbfs:/user/hive/warehouse/iws_state", True)

Out[1]: True

At the moment there are many variables in the table, maybe it would make sense to restructure the table somewhat. Also at the moment: do not need children_labels or children_id. What if this wasn't in one table, but two tables: iws_model and iws_children where iws_model had children id which matched the id-s in children table, so it would be one to many. This would create many rows which would be duplicates besides the id (or perhaps many-many). This would make updating the structure of the table more simple? If there is a need to change the structure of the model anyway. The same goes with having id-s as a string.

In [0]:
%sql
CREATE OR REPLACE TABLE iws_model AS
SELECT "root" AS id, "-" AS label, array("A") AS children_labels, array("root-A") AS children_id, 
    array(
        struct("root-A" AS node_id, "A" AS label, 1 as level, array() as events_between),
        struct("root-A-B" AS node_id, "B" AS label, 2 as level, array("A") as events_between),
        struct("root-A-B-X" AS node_id, "X" AS label, 3 as level, array("A","B") as events_between), 
        struct("root-A-B-D" AS node_id, "D" AS label, 3 as level, array("A","B") as events_between)
    ) AS nth_level_children
UNION ALL 
SELECT "root-A", "A", array("B"), array("root-A-B"), 
    array(
        struct("root-A-B" AS node_id, "B" AS label, 2 as level, array() as events_between),
        struct("root-A-B-X" AS node_id, "X" AS label, 3 as level, array("B") as events_between),
        struct("root-A-B-D" AS node_id, "D" AS label, 3 as level, array("B") as events_between),
        struct("root-A-B-D-C" AS node_id, "C" AS label, 4 as level, array("B","D") as events_between)
    )
UNION ALL 
SELECT "root-A-B", "B", array("X", "D"), array("root-A-B-X", "root-A-B-D"), 
    array(
        struct("root-A-B-X" AS node_id, "X" AS label, 3 as level, array() as events_between), -- "A-B-C" now "A-B-X"
        struct("root-A-B-D" AS node_id, "D" AS label, 3 as level, array() as events_between),
        struct("root-A-B-D-C" AS node_id, "C" AS label, 4 as level, array("D") as events_between)
    )
UNION ALL 
SELECT "root-A-B-X", "X", array(), array(),
    array()
UNION ALL 
SELECT "root-A-B-D", "D", array("C"), array("root-A-B-D-C"), 
    array(
        struct("root-A-B-D-C" AS node_id, "C" AS label, 4 as level, array() as events_between),
        struct("root-A-B-D-C-D" AS node_id, "D" AS label, 5 as level, array("C") as events_between),
        struct("root-A-B-D-C-C" AS node_id, "C" AS label, 5 as level, array("C") as events_between)
    )
UNION ALL 
SELECT "root-A-B-D-C", "C", array("D", "C"), array("root-A-B-D-C-D", "root-A-B-D-C-C"), 
    array(
        struct("root-A-B-D-C-D" AS node_id, "D" AS label, 5 as level, array() as events_between),
        struct("root-A-B-D-C-C" AS node_id, "C" AS label, 5 as level, array() as events_between)
    )
UNION ALL 
SELECT "root-A-B-D-C-D", "D", array(), array(), array()
UNION ALL 
SELECT "root-A-B-D-C-C", "C", array(), array(), array();

CREATE OR REPLACE TABLE iws_event 
SELECT "A" event, CURRENT_TIMESTAMP() ts, "trace_id_0" trace_id,;

CREATE OR REPLACE TABLE iws_state
(trace_id STRING, ts TIMESTAMP, current_node STRING,current_id STRING,cost_of_alignment INTEGER,previous_events STRING, trace STRING, execution_sequence STRING,event_level INTEGER,current_event_level INTEGER,current_node_level INTEGER);
--event level to filter out the latest alignments later


In [0]:
%sql
SELECT 
  explode(
    transform(
      filter(nth_level_children, x -> x.label = 'C'),
      x -> struct(x.node_id AS node_id, concat(repeat(">>", 7 - x.level), "-", "event") AS description)
    )
  ) AS nodes_with_label_C
FROM iws_model

In [0]:
import random 

activities = ["A","B","C","D","E","F"]
trace_id_lower = 0
trace_id_upper = 9

def insert_event(event=None, trace_id=None):
    if not event:
        event = random.choice(activities)
    if not trace_id:
        trace_id = 0 #random.randint(trace_id_lower, trace_id_upper)
    spark.sql(f"INSERT INTO iws_event SELECT '{event}', CURRENT_TIMESTAMP(), 'trace_id_{trace_id}'")

In [0]:
import pyspark.sql.functions as F 
from time import sleep
event_df = spark.readStream.table("iws_event").withWatermark("ts", "30 minutes")

In [0]:
event_df.createOrReplaceTempView("events")
#maybe add level already to events so dont have to take substring?

In [0]:

%sql
CREATE OR REPLACE TEMP VIEW latest_state AS 
SELECT trace_id, ts, current_node, current_id, cost_of_alignment,true_previous_events as previous_events,trace,execution_sequence,event_level, max_event_level as current_event_level,current_node_level FROM (
SELECT *, row_number() OVER (PARTITION BY trace_id,current_id order by event_level desc, cost_of_alignment asc) rn,
max(event_level) OVER (PARTITION BY trace_id) AS max_event_level, max(previous_events) OVER (PARTITION BY trace_id) as true_previous_events FROM iws_state
) WHERE rn = 1 --and event_level > current_event_level -2
--need to only get the values where when the node_id is the same, then the one with the highest event level is saved
-- Trace needs to be like this since we are possibly using past events (which are not updated) in the future

In [0]:
#if looking at older events, cost should be difference in event levels also number of skips >> between is same number
#TODO FIX IF EVENT IS FIRST COMING IN (fix with adding root row automatically)
#current level -> used to track what event the state is from if the state is used in the future
#event level -> used to track the current event level (how many events there have been)
#current_node_level -> used to track what level the node is, used in calculation
#model moves in trace(sigma), log moves in execution sequence (pi) 
#right now log and model moves are done without any difference really
spark.sql("""
SELECT trace_id, ts, exploded.*, previous_events
FROM (
    SELECT 
      e.trace_id, 
      e.ts,
      concat(e.previous_events,e.event) as previous_events,
      CASE 
        WHEN array_contains(transform(m.nth_level_children, x -> x.label), e.event) THEN
          transform(
            filter(m.nth_level_children, x -> x.label LIKE e.event),
            x -> struct(e.event as current_node, x.node_id as current_id, e.cost_of_alignment + abs(x.level - current_node_level-1) + current_event_level - event_level as cost_of_alignment, CASE WHEN current_event_level != event_level THEN concat(coalesce(trace, "root"),substring(previous_events,event_level+1,current_event_level+1), repeat(">>", abs(x.level - current_node_level-1)), e.event) ELSE concat(coalesce(trace,"root"),repeat(">>",abs(x.level - current_node_level-1)),e.event) END as trace,CONCAT(execution_sequence,repeat(">>", abs(current_event_level - event_level)), coalesce(CONCAT_WS('', x.events_between)),e.event) as execution_sequence, current_event_level+1 as event_level, current_event_level + 1 as current_event_level,x.level as current_node_level)
          )
        ELSE
          array(struct(e.current_node as current_node, m.id as current_id, e.cost_of_alignment + e.current_event_level - e.event_level + 1 as cost_of_alignment, concat(trace,substring(previous_events,event_level+1,current_event_level+1), e.event) as trace, concat(execution_sequence, repeat(">>", current_event_level - event_level), ">>")as execution_sequence, e.current_event_level + 1 as event_level, current_event_level + 1 as current_event_level, current_node_level))
      END AS exploded_struct
    FROM 
      (SELECT e.*, COALESCE(r.current_node, "-") as current_node, COALESCE(r.current_id, "root") as current_id, coalesce(cost_of_alignment, 0) as cost_of_alignment, coalesce(r.previous_events,"") as previous_events , coalesce(trace,"") as trace, coalesce(execution_sequence,"") as execution_sequence, coalesce(event_level, 0) as event_level, coalesce(current_event_level, 0) as current_event_level,coalesce(current_node_level,0) as current_node_level FROM events e LEFT JOIN latest_state r ON e.trace_id = r.trace_id) e 
    JOIN iws_model m ON e.current_id = m.id
) 
LATERAL VIEW explode(exploded_struct) t AS exploded
""").writeStream.format("delta").outputMode("append").option("checkpointLocation","/tmp/delta/state_append_32/").toTable("iws_state")

Out[28]: <pyspark.sql.streaming.query.StreamingQuery at 0x7fae842ab7f0>

In [0]:
%sql
SELECT * FROM latest_state where trace_id = "trace_id_6"
-- vaata mis state on

trace_id,ts,current_node,current_id,cost_of_alignment,previous_events,trace,execution_sequence,event_level,current_event_level,current_node_level
trace_id_6,2024-04-11T19:34:31.332+0000,A,root-A,2,AXFDXC,AXF,A>>>>,3,6,1
trace_id_6,2024-04-11T19:35:05.516+0000,D,root-A-B-D,4,AXFDXC,AXF>>DX,A>>>>BD>>,5,6,3
trace_id_6,2024-04-11T19:35:34.522+0000,C,root-A-B-D-C,4,AXFDXC,AXF>>DXC,A>>>>BD>>C,6,6,4
trace_id_6,2024-04-11T19:35:34.522+0000,C,root-A-B-D-C-C,5,AXFDXC,AXF>>DX>>C,A>>>>BD>>CC,6,6,5
trace_id_6,2024-04-11T19:35:34.522+0000,X,root-A-B-X,5,AXFDXC,A>>XFDXC,ABX>>>>>>>>,6,6,3


In [0]:
%sql
SELECT * from iws_state ---where trace_id = "trace_id_2" 

In [0]:
insert_event("A","6")
sleep(3)
insert_event("X","6")
sleep(5)
insert_event("F","6")
sleep(5)
insert_event("D","6")
sleep(5)
insert_event("X","6")
sleep(5)
insert_event("C","6")
#sleep(2)
#for i in range(5):
#    insert_event(trace_id = "5")
    #sleep(2)
# lisa uus event. Vaata uuesti mis state on (ülemine cell)
# võid katsetada lisada suvalist eventi vahepeal, nt X

In [0]:
#näited funktsioonide tööpõhimõtetest arusaamiseks

from pyspark.sql import SparkSession
from pyspark.sql.functions import expr

data = [
    (1, 'Event1', True, False),
    (2, 'Event2', True, True),
    (3, 'Event3', False, True)
]
columns = ['ID', 'Event', 'ConditionA', 'ConditionB']

df = spark.createDataFrame(data, schema=columns)

df.createOrReplaceTempView("events")

query = """
SELECT ID, Event, 
       explode(arrays_zip(
         array(CASE WHEN ConditionA THEN 'A' ELSE NULL END, CASE WHEN ConditionB THEN 'B' ELSE NULL END),
         array(CASE WHEN ConditionA THEN 'Condition A Met' ELSE NULL END, CASE WHEN ConditionB THEN 'Condition B Met' ELSE NULL END)
       )) as conditions
FROM events
WHERE ConditionA OR ConditionB
"""

result = spark.sql(query)

result = result.selectExpr("ID", "Event", "conditions['0'] as Condition", "conditions['1'] as Label")

result.show()

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, when, array, explode

data = [("A",), ("B",)]
schema = ["condition"]

df = spark.createDataFrame(data, schema)
df.show()

df.createOrReplaceTempView("conditions_table")

result_df = spark.sql("""
SELECT 
    exploded.item AS item,
    exploded.value AS value
FROM (
    SELECT explode(
        CASE 
            WHEN condition = 'A' THEN array(struct('A' AS item, 5 AS value))
            WHEN condition = 'B' THEN array(struct('D' AS item, 6 AS value), struct('C' AS item, 7 AS value), struct('F' AS item, 8 AS value))
            ELSE array() -- Handle other conditions or default case
        END
    ) AS exploded
    FROM conditions_table
) AS t
""")

result_df.show()